In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings 
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *
from statsforecast import StatsForecast
from statsforecast.models import Naive, HistoricAverage, WindowAverage, SeasonalNaive, AutoARIMA

warnings.filterwarnings("ignore")  # To ignore warnings from pandas/numpy

In [ ]:
user="Lilian"
#Read the panel
base_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\prebuilt_panels"
panel_df=pd.read_csv(os.path.join(base_path,"P1_nopop.csv"))

In [ ]:
# Filter to Indiana and sort by ATC2 Class and Period
df_in = panel_df[panel_df["State"] == "IN"].copy()

# Filter to period range 2017Q1 to 2024Q4
df_in = df_in[(df_in["Period"] >= "2017Q1") & (df_in["Period"] <= "2024Q4")]
df_in = df_in.sort_values(["ATC2 Class", "Period"]).reset_index(drop=True)

print(f"Indiana data shape: {df_in.shape}")
print(f"Period range: {df_in['Period'].min()} to {df_in['Period'].max()}")
print(f"Number of ATC2 classes: {df_in['ATC2 Class'].nunique()}")

In [ ]:
h = 4  # Number of quarters to forecast

# Define train-test split: training through 2023Q4, testing from 2024Q1 onward
train_end = "2023Q4"
test_start = "2024Q1"

# Define target variables
target_vars = ["Number of Prescriptions", "Units Reimbursed"]
atc2_classes = df_in["ATC2 Class"].unique()

print(f"Forecast horizon: {h} quarters")
print(f"Training period: up to {train_end}"); print(f"Test period: {test_start} onward")
print(f"Target variables: {target_vars}"); print(f"ATC2 classes to process: {len(atc2_classes)}")

In [ ]:
def historical_mean_forecast(train_values, h):
    #Forecast using the training mean repeated h times
    return np.full(h, train_values.mean())

def naive_forecast(train_values, h):
    #Forecast using the last observed training value repeated h times
    return np.full(h, train_values.iloc[-1])

def seasonal_naive_forecast(train_values, h, season_length=4):
    #Forecast using seasonal naive (repeat last season_length observations)
    last_season = train_values.iloc[-season_length:].values
    # Tile the last season to cover the forecast horizon
    n_full_cycles = h // season_length
    remainder = h % season_length
    forecast = np.tile(last_season, n_full_cycles)
    if remainder > 0:
        forecast = np.concatenate([forecast, last_season[:remainder]])
    return forecast

def autoarima_forecast(train_values, h):
    #Forecast using AutoARIMA (non-seasonal)
    # Prepare data in StatsForecast format
    train_df = pd.DataFrame({
        'unique_id': ['series'] * len(train_values),
        'ds': range(len(train_values)),
        'y': train_values.values
    })
    model = StatsForecast(models=[AutoARIMA(season_length=False, alias='ARIMA')], freq=1)
    model.fit(train_df)
    forecast_df = model.predict(h=h)
    return forecast_df['ARIMA'].values

def sarima_forecast(train_values, h, season_length=4):
    #Forecast using Seasonal AutoARIMA (SARIMA)
    # Prepare data in StatsForecast format
    train_df = pd.DataFrame({
        'unique_id': ['series'] * len(train_values),
        'ds': range(len(train_values)),
        'y': train_values.values
    })
    model = StatsForecast(models=[AutoARIMA(season_length=season_length, alias='SARIMA')], freq=1)
    model.fit(train_df)
    forecast_df = model.predict(h=h)
    return forecast_df['SARIMA'].values

def compute_mae(actual, predicted):
    #Compute Mean Absolute Error
    return np.mean(np.abs(actual - predicted))

In [ ]:
results = []

for target in target_vars:
    print(f"\n{'='*60}"); print(f"Processing target: {target}"); print(f"{'='*60}")
    
    skipped_count = 0
    processed_count = 0
    
    for atc2 in atc2_classes:
        # Extract series for this ATC2 class
        series_df = df_in[df_in["ATC2 Class"] == atc2][["Period", target]].copy()
        series_df = series_df.sort_values("Period").reset_index(drop=True)
        
        # Split into train and test
        train_df = series_df[series_df["Period"] <= train_end]
        test_df = series_df[series_df["Period"] >= test_start]
        
        # Skip if fewer than 24 training observations
        if len(train_df) < 24:
            skipped_count += 1
            continue
        
        # Skip if no test observations
        if len(test_df) == 0:
            skipped_count += 1
            continue
        
        train_values = train_df[target]
        test_values = test_df[target].values[:h]  # Limit to forecast horizon
        test_periods = test_df["Period"].values[:h]  # Limit to forecast horizon
        
        # Generate forecasts
        hist_mean_pred = historical_mean_forecast(train_values, h)
        naive_pred = naive_forecast(train_values, h)
        snaive_pred = seasonal_naive_forecast(train_values, h, season_length=4)
        arima_pred = autoarima_forecast(train_values, h)
        sarima_pred = sarima_forecast(train_values, h, season_length=4)
        
        # Compute MAE for each model
        hist_mean_mae = compute_mae(test_values, hist_mean_pred)
        naive_mae = compute_mae(test_values, naive_pred)
        snaive_mae = compute_mae(test_values, snaive_pred)
        arima_mae = compute_mae(test_values, arima_pred)
        sarima_mae = compute_mae(test_values, sarima_pred)
        
        # Store results with forecast and actual values for each period
        for i in range(h):
            actual = test_values[i]
            period = test_periods[i]
            
            results.append({
                "Target": target, "ATC2 Class": atc2, "Period": period,
                "Model": "Historical Mean", "Actual": actual, 
                "Forecast": hist_mean_pred[i], "Error": abs(actual - hist_mean_pred[i]),
                "MAE": hist_mean_mae
            })
            results.append({
                "Target": target, "ATC2 Class": atc2, "Period": period,
                "Model": "Naive", "Actual": actual,
                "Forecast": naive_pred[i], "Error": abs(actual - naive_pred[i]),
                "MAE": naive_mae
            })
            results.append({
                "Target": target, "ATC2 Class": atc2, "Period": period,
                "Model": "Seasonal Naive", "Actual": actual,
                "Forecast": snaive_pred[i], "Error": abs(actual - snaive_pred[i]),
                "MAE": snaive_mae
            })
            results.append({
                "Target": target, "ATC2 Class": atc2, "Period": period,
                "Model": "AutoARIMA", "Actual": actual,
                "Forecast": arima_pred[i], "Error": abs(actual - arima_pred[i]),
                "MAE": arima_mae
            })
            results.append({
                "Target": target, "ATC2 Class": atc2, "Period": period,
                "Model": "SARIMA", "Actual": actual,
                "Forecast": sarima_pred[i], "Error": abs(actual - sarima_pred[i]),
                "MAE": sarima_mae
            })
        
        processed_count += 1
    
    print(f"Processed: {processed_count} ATC2 classes")
    print(f"Skipped (insufficient data): {skipped_count} ATC2 classes")

# Create results DataFrame
results_df = pd.DataFrame(results)
print(f"\n{'='*60}"); print(f"Total results: {len(results_df)} rows"); print(f"{'='*60}")

In [ ]:
# Display the results table
print("Baseline Forecasting Results (Long Format)"); print("="*60)
results_df

In [ ]:
# Get unique MAE per Target, ATC2 Class, and Model (MAE is same for all periods within a group)
mae_df = results_df.groupby(["Target", "ATC2 Class", "Model"])["MAE"].first().reset_index()

# Summary statistics by Target and Model
summary = mae_df.groupby(["Target", "Model"])["MAE"].agg(["mean", "std", "min", "max"]).round(2)
summary.columns = ["Mean MAE", "Std MAE", "Min MAE", "Max MAE"]
print("Summary Statistics by Target and Model"); print("="*60)
summary

In [ ]:
# Identify best model per Target and ATC2 Class (using mae_df computed above)
best_models = mae_df.loc[mae_df.groupby(["Target", "ATC2 Class"])["MAE"].idxmin()]
best_models_display = best_models[["Target", "ATC2 Class", "Model", "MAE"]].sort_values(["Target", "ATC2 Class"]).reset_index(drop=True)
display(best_models_display)

# Count how often each model wins
print("\n")
model_wins = best_models.groupby(["Target", "Model"]).size().unstack(fill_value=0)
print("Model Wins (number of ATC2 classes where each model had lowest MAE):")
model_wins

In [ ]:
# Export results to CSV files

export_path = rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\exported_analysis"

# Export detailed results
results_df.to_csv(os.path.join(export_path, "forecast_traditional_IN_nopop.csv"), index=False)
print(f"Exported results_df to: {export_path}\\forecast_results_IN.csv")

# Export best models summary
best_models_display.to_csv(os.path.join(export_path, "best_models_IN_traditional_nopop.csv"), index=False)
print(f"Exported best_models_display to: {export_path}\\best_models_IN.csv")


Cross-Validation

In [ ]:
# Define models for cross-validation
models = [
    HistoricAverage(alias='Historical Mean'),
    Naive(alias='Naive'),
    SeasonalNaive(season_length=4, alias='Seasonal Naive'),
    AutoARIMA(season_length=False, alias='ARIMA')
]

cv_results = []

for target in target_vars:
    print(f"\n{'='*60}"); print(f"Cross-Validation for: {target}"); print(f"{'='*60}")
    
    skipped_count = 0
    processed_count = 0
    
    for atc2 in atc2_classes:
        # Extract series for this ATC2 class
        series_df = df_in[df_in["ATC2 Class"] == atc2][["Period", target]].copy()
        series_df = series_df.sort_values("Period").reset_index(drop=True)
        
        # Prepare data in StatsForecast format
        cv_df = pd.DataFrame({
            'unique_id': [f"{atc2}"] * len(series_df),
            'ds': range(len(series_df)),
            'y': series_df[target].values
        })
        
        # Create StatsForecast object
        sf = StatsForecast(models=models, freq=1, n_jobs=1)
        
        # Perform cross-validation
        cv_output = sf.cross_validation(
            df=cv_df,
            h=h,
            n_windows=5,
            step_size=h,
            refit=True
        )
        
        # Add metadata
        cv_output['Target'] = target
        cv_output['ATC2 Class'] = atc2
        
        cv_results.append(cv_output)
        processed_count += 1
    
    print(f"Processed: {processed_count} ATC2 classes")
    print(f"Skipped (insufficient data): {skipped_count} ATC2 classes")

# Combine all cross-validation results
if len(cv_results) > 0:
    cv_results_df = pd.concat(cv_results, ignore_index=True)
    print(f"\n{'='*60}"); print(f"Cross-validation complete! Total rows: {len(cv_results_df)}"); print(f"{'='*60}")
else:
    cv_results_df = pd.DataFrame()
    print("\nNo series had sufficient data for cross-validation.")

In [ ]:
# Display cross-validation results
print("Cross-Validation Results"); print("="*60)
display(cv_results_df)

# Compute MAE for each model across all cross-validation windows
model_columns = ['Historical Mean', 'Naive', 'Seasonal Naive', 'ARIMA']

# Melt the dataframe to long format for easier analysis
cv_long = cv_results_df.melt(
    id_vars=['unique_id', 'ds', 'cutoff', 'y', 'Target', 'ATC2 Class'],
    value_vars=model_columns,
    var_name='Model',
    value_name='Forecast'
)
cv_long['Error'] = np.abs(cv_long['y'] - cv_long['Forecast'])

# Compute MAE per Target, ATC2 Class, and Model
cv_mae = cv_long.groupby(['Target', 'ATC2 Class', 'Model'])['Error'].mean().reset_index()
cv_mae.columns = ['Target', 'ATC2 Class', 'Model', 'CV_MAE']

print("\nCross-Validation MAE by Target, ATC2 Class, and Model"); print("="*60)
display(cv_mae)

# Summary statistics by Target and Model
cv_summary = cv_mae.groupby(['Target', 'Model'])['CV_MAE'].agg(['mean', 'std', 'min', 'max']).round(2)
cv_summary.columns = ['Mean CV_MAE', 'Std CV_MAE', 'Min CV_MAE', 'Max CV_MAE']
print("\nCross-Validation Summary Statistics by Target and Model"); print("="*60)
display(cv_summary)

# Best model per Target and ATC2 Class based on CV
cv_best_models = cv_mae.loc[cv_mae.groupby(['Target', 'ATC2 Class'])['CV_MAE'].idxmin()]
cv_best_display = cv_best_models[['Target', 'ATC2 Class', 'Model', 'CV_MAE']].sort_values(['Target', 'ATC2 Class']).reset_index(drop=True)
print("\nBest Model per Target and ATC2 Class (Cross-Validation)"); print("="*60)
display(cv_best_display)

# Model wins based on cross-validation
cv_model_wins = cv_best_models.groupby(['Target', 'Model']).size().unstack(fill_value=0)
print("\nModel Wins (Cross-Validation):"); print("="*60)
cv_model_wins